<a href="https://colab.research.google.com/github/marialejandrapulido1/analitica-negocios/blob/main/Reto_2__MAPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Maria Alejandra Pulido**

Reto 2 - Árboles de decisión

*Cargar librerías de trabajo*

In [ ]:
#libreria que conecta con excel
import numpy as np
#libreria con funciones numericas
import pandas as pd

#implementacion y evaluacion del modelo
#para crear arbol de decision
from sklearn.tree import DecisionTreeClassifier
#para analizar modelo
from sklearn.metrics import confusion_matrix

#cargar automaticamente los archivos de drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


*Cargar base de datos*

In [ ]:
drive = '/content/drive/MyDrive/ANALITICA/2. TechSalesReport_Int_L.xlsx'
#XDB = X database
XDB = pd.read_excel(drive, sheet_name = 0)
#confirmar si es el archivo correcto
XDB.head(4)

#eliminar variable sales rep
XDB = XDB.drop(['Sales_Rep'], axis = 1)

#eliminar variable personality
XDB = XDB.drop(['Personality'], axis = 1)

#convertir business a valores numéricos
XDB['Business'] = XDB['Business'].map({'Hardware': 0, 'Software': 1})

#organizar college en yes - 1 y no - 0
XDB['College'] = XDB['College'].map({'No': 0, 'Yes': 1})

#seleccionar las variables que necesito
#vector con variables que necesito
XD = XDB[['Business', 'Age', 'Female', 'Years', 'Certificates', 'Feedback', 'Salary', 'NPS']]

#vector que quiero pronosticar - college
yd = XDB[['College']]

#revisar si quedo bien
XDB.head(4)

,Business,Age,Female,Years,College,Certificates,Feedback,Salary,NPS
0,0,59,1,2,1,1,2.01,70200,5
1,0,52,0,10,1,4,3.64,133000,10
2,1,47,1,1,1,1,3.88,52600,8
3,0,61,0,2,1,3,2.70,96000,6


**Explicación de variables**

- Business: Do they work in hardware (0) or software (1)
- Age: Age of the person
- Female: Is it a female? Yes (1), No (0)
- Years: Years working in the company
- College: Did they go to College: Yes (1), No (0)
- Certificates: Amount of certificates the employee has
- Feedback: Employee's score from HR
- Salary: How much the person earn
- NPS: Net Promoter Score (customer loyalty)


*Implementar el modelo*

In [ ]:
#arbol = arbol de decision
arbol = DecisionTreeClassifier(criterion = 'gini', max_depth = 4)
#max depth = profundidad del arbol es 4
#gini = criterio que indica grado de error del modelo
arbol.fit(XD, yd)

#desplegar arbol
#exportar
from sklearn.tree import export_graphviz
#crear grafico
from pydotplus import graph_from_dot_data

#etiquetar arbol
vs = ['Business', 'Age', 'Female', 'Years', 'Certificates', 'Feedback', 'Salary', 'NPS']

#dejar listo arbol para descargar
dot_data = export_graphviz(arbol, feature_names = vs)
grafico = graph_from_dot_data(dot_data)
grafico.write_png('collegearbol1.png')


True

**Breve descripción del árbol**
- En el arbol de decision hoy nay nodos puros.
- La variable más decisiva es Feedback < = 3.4, ya que tiene un gini de 0.075.

**Reglas del negocio ejemplos**
- If Salary < = 64100, and Salary < = 47,100, and Salary < = 35,100, and Age < = 23.5, la persona no tiene título universitario. Total de datos 51, 19 tienen título (37%) y 32 no tienen (63%).
- If Salary > 64100, and Salary > 96500, and Certificates > 4.5, and Salary > 109100, la persona si tiene título universitarioo. El total de datos es de 762, 57 no tienen título (7.4%) y 705 si tienen (92.6%).

*Evaluar desempeño del modelo*

In [ ]:
ydp = arbol.predict(XD)
cm = confusion_matrix(yd, ydp)
print(cm)

#matriz de confusion con verdaderos negativos 17451 y positivos 114
VN = cm [0][0]; FP = cm [0][1]; FN = cm [1][0]; VP = cm [1][1]

#exactitud = desempeño del modelo
exactitud = (VP + VN) / (VP + VN + FP + FN)
print('La exactitud con la que funciona el modelo en general es = ', exactitud)

#sensibilidad = como se comporta el modelo con los universitarios
sensibilidad = VP / (VP + FN)
print('La sensibilidad es = ', sensibilidad)

#especificidad = como se comporta el modelo prediciendo los no universitarios
especificidad = VN / (VN + FP)
print('La especificidad es = ', especificidad)

#precision = que tan preciso es el modelo clasificando universitarios
precision = VP / (VP + FP)
print('La precision es = ', precision)

#tasa de error = cuanto se equivoca el modelo
error = (FP + FN) / (VP + VN + FP + FN)
print('La tasa de error es = ', error)

#prediccion negativa = comportamiento del modelo identificando los no universitarios
prediccion_negativa = VN / (VN + FN)
print('La prediccion negativa es = ', prediccion_negativa)

#prediccion positiva
prediccion_positiva = VP / (VP + FP)
print('La prediccion positiva es = ', prediccion_positiva)

[[  114  4356]
 [   69 17451]]
La exactitud con la que funciona el modelo en general es =  0.7987721691678036
La sensibilidad es =  0.9960616438356165
La especificidad es =  0.025503355704697986
La precision es =  0.8002476269087908
La tasa de error es =  0.20122783083219645
La prediccion negativa es =  0.6229508196721312
La prediccion positiva es =  0.8002476269087908


**Descripción de resultados**

Se puede observar con los resultados obtenidos del modelo, que la exactitud y sensibilidad con la que este funciona es de 79% y 99%, respectivamente. A pesar de que el modelo cuenta con una precisión de 80% para clasificar a los universitarios, este tiene una tasa de error de 2%.

*Pronóstico*

In [ ]:
#tomar la totalidad de los solicitantes
XDB2 = XDB
#revisar
XDB2.head(4)

yd2 = XDB2[['College']]
#revisar
yd2.head(4)

#los que no tienen valores en college
filas = np.where((np.isnan(yd2)) == False) [1]
print("Filas sin informacion: ", filas)

#pronostico para esas personas
XD2 = XDB2[['Business', 'Age', 'Female', 'Years', 'Certificates', 'Feedback', 'Salary', 'NPS']]
XD2 = XD2.iloc[filas]

ydp2 = arbol.predict(XD2)
print("Predicciones: ", ydp2)

Filas sin informacion:  [0 0 0 ... 0 0 0]
Predicciones:  [1 1 1 ... 1 1 1]


*Crear informe*

In [ ]:
dataframepro = pd.DataFrame(np.column_stack((XD2, ydp2)))
#añadir college
dataframepro.columns = ['Business', 'Age', 'Female', 'Years', 'Certificates', 'Feedback', 'Salary', 'NPS', 'College']
dataframepro.head()
dataframepro.to_excel('Informe1.xlsx')

print("Negados pronosticados: ", len(np.where(ydp2 == 0)[0]))
print("Aprobados pronosticados: ", len(np.where(ydp2 == 1)[0]))


Negados pronosticados:  0
Aprobados pronosticados:  21990


In [ ]:
#filas sin informacion en college
filas = XDB[XDB['College'].isna()].index
print("Filas sin informacion en college: ", filas)

#coger las variables para predecir las filas
XD2 = XDB.loc[filas, ['Business', 'Age', 'Female', 'Years', 'Certificates', 'Feedback', 'Salary', 'NPS']]
XD2 = XDB[['Business', 'Age', 'Female', 'Years', 'Certificates', 'Feedback', 'Salary', 'NPS']]

#prediccion modelo
ydp2 = arbol.predict(XD2)

#dataframe de pronostico con predicciones
dataframepro = XD2.copy()
dataframepro['CollegePrecitions'] = ydp2

#guardar excel
dataframepro.to_excel('Informe1.xlsx', index = False)

#resultados de pronostico
print("Negados pronosticados: ", (ydp2 == 0).sum())
print("Aprobados pronosticados: ", (ydp2 == 1).sum())

Filas sin informacion en college:  Index([], dtype='int64')
Negados pronosticados:  183
Aprobados pronosticados:  21807


In [ ]:
#prediccion
XD2 = np.array(XDB2[['Business','Age', 'Female', 'Years', 'Certificates', 'Feedback', 'Salary', 'NPS']])
print(XDB2)

ydp2  = arbol.predict(XD2)
print(ydp2)

#resultados prediccion
print("Numero de personas inscritas es: ", np.sum(ydp2))
print("Nmero total de personas es: ", len(ydp2))
print("Numero de personas no inscritas es: ", len(ydp2)-np.sum(ydp2))

#poner en excel
dataframepro = pd.DataFrame(np.column_stack((XD2, ydp2)))
dataframepro.columns = ['Filas', 'Business', 'Age', 'Female', 'Years', 'Certificates', 'Feedback', 'Salary', 'NPS']
dataframepro.head()
dataframepro.to_excel('Informe1.xlsx')

       Business  Age  Female  Years  College  Certificates  Feedback  Salary  \
0             0   59       1      2        1             1      2.01   70200   
1             0   52       0     10        1             4      3.64  133000   
2             1   47       1      1        1             1      3.88   52600   
3             0   61       0      2        1             3      2.70   96000   
4             1   39       0      1        0             5      3.44  122000   
...         ...  ...     ...    ...      ...           ...       ...     ...   
21985         1   44       0      1        1             4      1.80  128000   
21986         1   23       1      6        1             6      1.77   53400   
21987         0   48       1      4        1             0      2.46   50400   
21988         1   29       0      4        1             2      3.68   70000   
21989         1   23       1      2        1             1      2.13   47400   

       NPS  
0        5  
1       10  


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


*Pronostico*

In [ ]:
#total de solicitantes
XDB2 = pd.read_excel(drive, sheet_name = 1)
XDB2.head(4)

#organizar hardware y software en valores numericos
XDB2['Business'] = XDB2['Business'].map({'Hardware': 0, 'Software': 1})

#crear college con valores vaciones
XDB2['College'] = np.nan
print(XDB2)

yd2 = XDB2[['College']]
yd2.head(4)

#filas sin aprobacion
filas = np.where((np.isnan(yd2)) == True)[0]
print("Filas sin aprobacion: ", filas)

#pronostico preaprobacion para estas personas
XD2 = XDB2  [['Business', 'Age', 'Female', 'Years', 'Certificates', 'Feedback', 'Salary', 'NPS']]
XD2 = XD2.iloc[filas,]
ydp2 = arbol.predict(XD2)
print("Predicciones: ", ydp2)

#informe con preaprobacion
dataframepro = pd.DataFrame(np.column_stack((filas, XD2, ydp2)))
dataframepro.columns = ['Filas', 'Business', 'Age', 'Female', 'Years', 'Certificates', 'Feedback', 'Salary', 'NPS', 'College']
dataframepro.head(4)
dataframepro.to_excel('Informe1.xlsx')

#resultados
print("Negados pronosticados: ", len(np.where(ydp2 == 0)[0]))
print("Aprobados pronosticados: ", len(np.where(ydp2 == 1)[0]))

    Sales_Rep  Business  Age  Female  Years Personality  Certificates  \
0           1         0   63       0      2    Sentinel             5   
1           2         1   38       1      2    Diplomat             4   
2           3         1   47       0      1     Analyst             2   
3           4         1   25       0      1    Explorer             2   
4           5         0   49       0     10    Explorer             4   
5           6         1   48       0      1    Diplomat             1   
6           7         1   25       1      1    Explorer             6   
7           8         0   39       0      1    Diplomat             2   
8           9         1   29       0      1    Explorer             5   
9          10         1   45       1      3    Diplomat             3   
10         11         0   35       0      6    Diplomat             1   
11         12         0   30       0      4    Diplomat             2   
12         13         1   28       0      1     Ana

# **Análisis de resultados**

Para la implementación del modelo de arbol de decisión, se tomaron un total de 9 variables, 8 variables socioeconómicas de entrada en las que se destacan negocios, edad, femenino, años, certificados, retroalimentación, salario y NPS; y una variable de salida que le permite evaluar si un empleado tiene título universitario o no.

Para la construccion del modelo se tuvieron un total de 21,990 datos divididos en 4,470 que no tienen título universitario y un total de 17,420 que si tienen. Luego de la ejecución del modelo se logro una exactitud cercana al 79% en promedio, lo que esta por encima del 75% frente a la exactitud, lo que indica la calidad del modelo en el pronostico.

Es importante destacar que el coeficiente de sensibilidad estuvo por encima del 60% (99%) mientras que la prediccion negativa se ubico igualmente por encima del 60% (62%), lo que corrobora el buen funcionamiento general del modelo.

La base de datos completa posee un total de 928 personas sin título universitario. Luego de la construccion del modelo, se procedio con el pronostico del valor de título universitario de las personas. Luego se logro un informe de estas personas en donde un total de 923 (99%) no tienen título universitario y un total de 5 si tienen título (1%).

# **Cuestionario**



In [ ]:
#pregunta 1
#determinar la cantidad total de datos usados en el modelo
datos = print(f"Cantidad de datos utilizados: {XDB.shape[0]}")
#intervalo de 20,000 a 22,000 (21,990)

#pregunta 2
#intervalo en el cual se encuentra el número de personas que no poseen título universitario IF Salario<=64100 and Salary>47100 and Age>39.5 and Certificates<=1.5
#segun el arbol de decision son 356 no tienen titulo y 1,048 si tienen.
#intervalo de 350 a 400

#pregunta 3
#cantidad de nodos puros
#en el modelo no hay nodos puros
#intervalo de 0 a 5

#pregunta 4
#intervalo con titulo pronosticado de yes
print("Aprobados pronosticados: ", len(np.where(ydp2 == 1)[0]))
#intervalo de 15 a 20 (16)

#pregunta 5
#intervalo de especificidad
especificidad = VN / (VN + FP)
print('La especificidad es = ', especificidad)
#intervalo de 0 a 5 (2.5%)

#pregunta 6
#intervalo de verdaderos negativos
VN = cm [0][0]
VN
#intervalo de 100 a 120 (114)

#pregunta 7
#intervalo de exactitud
exactitud = (VP + VN) / (VP + VN + FP + FN)
print('La exactitud con la que funciona el modelo en general es = ', exactitud)
#intervalo de 75 a 80 (79%)

#pregunta 8
#verdaderos positivos
VP = cm [1][1]
VP
#intervalo de 17,000 a 17,500 (17,451)

#pregunta 9
#intervalo precision
precision = VP / (VP + FP)
print('La precision es = ', precision)
#intervalo de 80 a 85 (80%)

#pregunta 10
#personas que poseen un titulo universitario para la siguiente regla IF Salario<=64100 and Salary>47100 and Age>39.5 and Certificates>1.5
#segu el arbol de decision 986 personas si tienen titulo
#intervalo de 950 a 1000 (986)

Cantidad de datos utilizados: 21990
Aprobados pronosticados:  16
La especificidad es =  0.025503355704697986
La exactitud con la que funciona el modelo en general es =  0.7987721691678036
La precision es =  0.8002476269087908
